Setup: Клонирование репозитория и загрузка чекпоинтов

In [ ]:
# Ячейка 1: Setup

import os
import subprocess
import glob
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth  # Если запускаете в Colab; для локального запуска удалите
from oauth2client.client import GoogleCredentials

# Функция клонирования репозитория, если папка еще не существует
def clone_repo_if_not_exists(repo_url, dest_folder):
    if os.path.exists(dest_folder):
        print(f"[INFO] Репозиторий уже существует: {dest_folder}")
        return
    print(f"[INFO] Клонирование репозитория: {repo_url}")
    subprocess.run(['git', 'clone', repo_url, dest_folder], check=True)
    print(f"[INFO] Репозиторий успешно клонирован в: {dest_folder}")

# Функция для скачивания файла с Google Drive, если его еще нет
def download_from_google_drive(file_id, file_dst):
    if os.path.exists(file_dst):
        print(f"[INFO] Файл уже существует: {file_dst}")
        return
    print(f"[INFO] Скачивание файла в: {file_dst}")
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(file_dst)
    print(f"[INFO] Файл успешно скачан: {file_dst}")

# Аутентификация в Google Drive (в Colab требуется, даже если папка публичная)
auth.authenticate_user()  # Если запускаете в Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Клонируем репозиторий HairMapper, если он еще не клонирован
REPO_URL = "https://github.com/Lunatik-006/HairMapper.git"
REPO_FOLDER = "./HairMapper"
clone_repo_if_not_exists(REPO_URL, REPO_FOLDER)

# Переходим в папку репозитория
os.chdir(REPO_FOLDER)
print("[INFO] Текущая рабочая директория:", os.getcwd())

# ==================== Загрузка чекпоинтов моделей ====================
# Словарь с данными для чекпоинтов: имя файла, ID Google Drive и папка назначения
checkpoints = {
    'StyleGAN2-ada-Generator.pth': {
        'url': '1EsGehuEdY4z4t21o2LgW2dSsyN3rxYLJ',
        'dir': './ckpts'
    },
    'e4e_ffhq_encode.pt': {
        'url': '1cUv_reLE6k3604or78EranS7XzuVMWeO',
        'dir': './ckpts'
    },
    'model_ir_se50.pth': {
        'url': '1GIMopzrt2GE_4PG-_YxmVqTQEiaqu5L6',
        'dir': './ckpts'
    },
    'face_parsing.pth': {
        'url': '1IMsrkXA9NuCEy1ij8c8o6wCrAxkmjNPZ',
        'dir': './ckpts'
    },
    'vgg16.pth': {
        'url': '1EPhkEP_1O7ZVk66aBeKoFqf3xiM4BHH8',
        'dir': './ckpts'
    }
}

for ckpt_name, info in checkpoints.items():
    output_dir = info['dir']
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, ckpt_name)
    download_from_google_drive(file_id=info['url'], file_dst=output_path)

# Загрузка чекпоинтов классификаторов (для gender/hair)
classification_ckpt = [
    {'url': '1SSw6vd-25OGnLAE0kuA-_VHabxlsdLXL', 'dir': './classifier/gender_classification'},
    {'url': '1n14ckDcgiy7eu-e9XZhqQYb5025PjSpV', 'dir': './classifier/hair_classification'}
]
for clf in classification_ckpt:
    output_dir = clf['dir']
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'classification_model.pth')
    download_from_google_drive(file_id=clf['url'], file_dst=output_path)


Установка пакетов и зависимостей

In [ ]:
# Ячейка 2: Установка пакетов
# Если запускаете в Colab, используйте команды !pip install и !wget.
# В локальной среде выполните установку через командную строку или Python.

# Установка PyTorch с поддержкой CUDA (если у вас NVIDIA GPU) или CPU-версия
!pip install torch===2.0.0+cu117 torchvision===0.15.0+cu117 torchaudio===2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

# Установка остальных зависимостей из requirements.txt
!pip install -r requirements.txt

# Понижение версии Pillow до 9.5.0 для совместимости с torchvision и numpy<2.0 для совместимости с pillow
!pip install pillow==9.5.0
!pip install "numpy<2.0"

# Установка Ninja (для сборки CUDA расширений, если требуется)
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force


3. Загрузка модели mapper и тестового изображения

In [ ]:
# Ячейка 3: Загрузка предобученной модели mapper и тестового изображения

# Загрузка модели mapper (папка: ./mapper/checkpoints/final)
mapper_url = 'https://drive.google.com/file/d/1F3oujXbvalqEOixcAkIyURuY512nmroe'
# Преобразуем URL для получения ID
mapper_id = mapper_url.replace('https://drive.google.com/file/d/', '').split('/')[0]
mapper_output_dir = './mapper/checkpoints/final'
os.makedirs(mapper_output_dir, exist_ok=True)
mapper_output_path = os.path.join(mapper_output_dir, 'best_model.pt')
download_from_google_drive(file_id=mapper_id, file_dst=mapper_output_path)

# Загрузка тестового изображения (сохранится в ./test_data/origin)
test_img_name = '00010.png'
test_img_id = '1f-cHWMczIyjYBWRnypi1brOpFf2skgWd'
test_img_dir = './test_data/origin'
os.makedirs(test_img_dir, exist_ok=True)
test_img_path = os.path.join(test_img_dir, test_img_name)
download_from_google_drive(file_id=test_img_id, file_dst=test_img_path)


4. Переход в папку encoder4editing, Импорт модулей и загрузка модели pSp (энкодер e4e)

In [ ]:
# Переход в папку encoder4editing
# (Убедитесь, что папка encoder4editing присутствует в репозитории или клонируйте её, если требуется)
os.chdir('./encoder4editing')
print("[INFO] Текущая директория (encoder4editing):", os.getcwd())
# Импорт модулей и загрузка модели pSp (энкодер e4e)
from argparse import Namespace
import sys
import torch
import torchvision.transforms as transforms
import numpy as np
import PIL.Image
from PIL import ImageFile
import glob
import argparse

sys.path.append(".")
sys.path.append("..")
ImageFile.LOAD_TRUNCATED_IMAGES = True

from models.psp import pSp  # Импорт модели pSp

# Определяем трансформации для входного изображения
img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Загружаем чекпоинт для энкодера e4e (файл из ./ckpts)
model_path = "../ckpts/e4e_ffhq_encode.pt"
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print("[INFO] Модель pSp (энкодер e4e) загружена.")


5. Кодирование изображений: создание латентных кодов для тестовых изображений

In [ ]:
# Ячейка 5: Кодирование изображений (из папки с исходными изображениями в папку с латентными кодами)
# Исходные изображения находятся в папке: ../test_data/origin
# Латентные коды будут сохраняться в папке: ../test_data/code

data_dir = '../test_data'
origin_dir = os.path.join(data_dir, 'origin')  # Папка с исходными изображениями (отсюда вы можете вручную проверять изображения)
code_dir = os.path.join(data_dir, 'code')        # Папка для сохранения латентных кодов
os.makedirs(code_dir, exist_ok=True)

def run_on_batch(inputs, net):
    # Функция для получения латентного представления
    latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
    return latents

# Обработка изображений: для каждого файла (png/jpg) из origin_dir,
# если соответствующий файл с латентным кодом отсутствует в code_dir, создаем его.
for file_path in glob.glob(os.path.join(origin_dir, '*.png')) + glob.glob(os.path.join(origin_dir, '*.jpg')):
    name = os.path.basename(file_path)[:-4]
    code_path = os.path.join(code_dir, f'{name}.npy')
    if os.path.exists(code_path):
        print(f"[INFO] Латентный код уже существует: {code_path}")
        continue
    input_image = PIL.Image.open(file_path).convert('RGB')
    transformed_image = img_transforms(input_image)
    with torch.no_grad():
        latents = run_on_batch(transformed_image.unsqueeze(0), net)
        latent = latents[0].cpu().numpy()
        latent = np.reshape(latent, (1, 18, 512))
        np.save(code_path, latent)
        print(f"[INFO] Латентный код сохранен: {code_path}")


6. Запуск удаления волос с изображения с использованием mapper и генератора StyleGAN2-ada

In [ ]:
# Ячейка 6: Удаление волос (выполнение стиля смешивания с mapper)
# Входные данные:
# - Латентные коды из папки: ./test_data/code
# - Исходные изображения из папки: ./test_data/origin
# Результаты сохраняются в папку: ./test_data/mapper_res

os.chdir('../')  # Возвращаемся в корневую папку репозитория
print("[INFO] Текущая директория:", os.getcwd())

import cv2
import argparse
from styleGAN2_ada_model.stylegan2_ada_generator import StyleGAN2adaGenerator
from tqdm import tqdm
from classifier.src.feature_extractor.hair_mask_extractor import get_hair_mask, get_parsingNet
from mapper.networks.level_mapper import LevelMapper
import torch
import glob
import numpy as np
from PIL import Image
from diffuse.inverter_remove_hair import InverterRemoveHair  # если используется

# Параметры моделей и путей
model_name = 'stylegan2_ada'
latent_space_type = 'wp'
data_dir = './test_data'
origin_img_dir = os.path.join(data_dir, 'origin')    # Исходные изображения (для ручной проверки – см. папку)
code_dir = os.path.join(data_dir, 'code')              # Латентные коды (см. папку)
res_dir = os.path.join(data_dir, 'mapper_res')         # Результаты обработки будут сохраняться здесь
os.makedirs(res_dir, exist_ok=True)

# Инициализация генератора StyleGAN2-ada
print("[INFO] Инициализация генератора...")
model = StyleGAN2adaGenerator(model_name, logger=None, truncation_psi=1.0)

# Инициализация mapper (удаление волос)
mapper = LevelMapper(input_dim=512).eval().cuda()
ckpt = torch.load('./mapper/checkpoints/final/best_model.pt')
alpha = float(ckpt['alpha']) * 1.2  # Коэффициент изменения
mapper.load_state_dict(ckpt['state_dict'], strict=True)
kwargs = {'latent_space_type': latent_space_type}

# Загрузка модели для парсинга лица (используется для извлечения маски волос)
parsingNet = get_parsingNet(save_pth='./ckpts/face_parsing.pth')

# Если используется дополнительная инверсия для удаления волос (можно отключить, если не нужно)
inverter = InverterRemoveHair(
    model_name,
    Generator=model,
    learning_rate=0.01,
    reconstruction_loss_weight=1.0,
    perceptual_loss_weight=5e-5,
    truncation_psi=1.0,
    logger=None
)

# Процесс обработки латентных кодов
code_list = glob.glob(os.path.join(code_dir, '*.npy'))
total_num = len(code_list)
print(f"[INFO] Обработка {total_num} образцов.")
pbar = tqdm(total=total_num)
for code_path in code_list:
    pbar.update(1)
    name = os.path.basename(code_path)[:-4]
    # Определяем путь к исходному изображению (png или jpg)
    f_path_png = os.path.join(origin_img_dir, f'{name}.png')
    f_path_jpg = os.path.join(origin_img_dir, f'{name}.jpg')
    if os.path.exists(os.path.join(res_dir, f'{name}.png')):
        continue
    if os.path.exists(f_path_png):
        origin_img_path = f_path_png
    elif os.path.exists(f_path_jpg):
        origin_img_path = f_path_jpg
    else:
        continue

    # Загрузка латентного кода и его преобразование
    latent_codes_origin = np.reshape(np.load(code_path), (1, 18, 512))
    mapper_input = latent_codes_origin.copy()
    mapper_input_tensor = torch.from_numpy(mapper_input).cuda().float()
    edited_latent_codes = latent_codes_origin
    edited_latent_codes[:, :8, :] += alpha * mapper(mapper_input_tensor).to('cpu').detach().numpy()

    # Загрузка исходного изображения (для проверки – см. папку origin)
    origin_img = cv2.imread(origin_img_path)

    # Генерация нового изображения с помощью функции easy_style_mixing
    outputs = model.easy_style_mixing(latent_codes=edited_latent_codes,
                                      style_range=range(7, 18),
                                      style_codes=latent_codes_origin,
                                      mix_ratio=0.8,
                                      **kwargs)
    edited_img = outputs['image'][0][:, :, ::-1]  # Перевод из BGR в RGB

    # Получаем маску волос (сохраненные результаты можно проверить в папке с исходными изображениями)
    hair_mask = get_hair_mask(img_path=origin_img, net=parsingNet, include_hat=True, include_ear=True)
    mask_dilate = cv2.dilate(hair_mask, kernel=np.ones((50, 50), np.uint8))
    mask_dilate_blur = cv2.blur(mask_dilate, ksize=(30, 30))
    mask_dilate_blur = (hair_mask + (255 - hair_mask) / 255 * mask_dilate_blur).astype(np.uint8)
    face_mask = 255 - mask_dilate_blur
    face_mask = cv2.resize(face_mask, (origin_img.shape[1], origin_img.shape[0]))

    # Вычисление центра области для seamlessClone (на основе маски)
    idx = np.where(face_mask > 0)
    cy = (np.min(idx[0]) + np.max(idx[0])) // 2
    cx = (np.min(idx[1]) + np.max(idx[1])) // 2
    center = (cx, cy)

    res_save_path = os.path.join(res_dir, f'{name}.png')
    # Применяем seamlessClone для аккуратного объединения областей
    mixed_clone = cv2.seamlessClone(origin_img, edited_img, face_mask[:, :, 0], center, cv2.NORMAL_CLONE)
    cv2.imwrite(res_save_path, mixed_clone)
pbar.close()
print("[INFO] Обработка завершена.")


7. Визуализация результатов

In [ ]:
# Ячейка 7: Визуализация результатов
from IPython.display import display
import cv2
import glob
import numpy as np
from PIL import Image

# Вывод результатов обработки. Изображения сохраняются в папке: ./test_data/mapper_res
for res_path in glob.glob('./test_data/mapper_res/*'):
    res_img = cv2.imread(res_path)[:, :, ::-1]  # Конвертация из BGR в RGB
    res_im = Image.fromarray(res_img)
    display(res_im)
